# Dynamic Programming for Reinforcement Learning

**Chapter 3: Model-Based Planning and Optimal Control**

Dynamic programming (DP) provides a collection of algorithms for computing optimal policies when we have a perfect model of the environment as a Markov Decision Process (MDP). While DP methods are limited by their assumption of a perfect model and high computational cost, they provide the theoretical foundation for understanding reinforcement learning.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns

np.random.seed(42)
sns.set_style('whitegrid')

In [ ]:
class GridWorld:
    """
    GridWorld MDP environment.
    
    Grid layout:
    - 'S': Start state
    - 'G': Goal state (+10 reward)
    - 'X': Obstacle (blocked)
    - '.': Normal state (-1 step cost)
    """
    
    def __init__(self, grid_size=(5, 5), goal=(4, 4), obstacles=None, gamma=0.9):
        """
        Args:
            grid_size: (height, width) of grid
            goal: (row, col) of goal state
            obstacles: list of (row, col) obstacle positions
            gamma: discount factor
        """
        self.height, self.width = grid_size
        self.goal = goal
        self.obstacles = obstacles or []
        self.gamma = gamma
        
        # Actions: up, right, down, left
        self.actions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
        self.action_names = ['→', '↓', '←', '↑']
        self.n_actions = len(self.actions)
        
        # Generate all states
        self.states = []
        for i in range(self.height):
            for j in range(self.width):
                if (i, j) not in self.obstacles:
                    self.states.append((i, j))
        
        self.n_states = len(self.states)
        self.state_to_idx = {s: i for i, s in enumerate(self.states)}
    
    def is_terminal(self, state):
        """Check if state is terminal (goal)."""
        return state == self.goal
    
    def get_next_state(self, state, action_idx):
        """Get next state given current state and action."""
        if self.is_terminal(state):
            return state  # Terminal state stays terminal
        
        action = self.actions[action_idx]
        next_state = (state[0] + action[0], state[1] + action[1])
        
        # Check boundaries and obstacles
        if (0 <= next_state[0] < self.height and 
            0 <= next_state[1] < self.width and 
            next_state not in self.obstacles):
            return next_state
        else:
            return state  # Invalid move, stay in place
    
    def get_reward(self, state, action_idx, next_state):
        """Get reward for transition."""
        if next_state == self.goal:
            return 10.0  # Goal reward
        else:
            return -1.0  # Step cost
    
    def get_transition_prob(self, state, action_idx, next_state):
        """
        Get transition probability p(s'|s,a).
        For deterministic GridWorld, this is 1.0 for the resulting state.
        """
        expected_next = self.get_next_state(state, action_idx)
        return 1.0 if next_state == expected_next else 0.0

# Create GridWorld with obstacles
env = GridWorld(
    grid_size=(5, 5),
    goal=(4, 4),
    obstacles=[(1, 1), (2, 2), (3, 1)],
    gamma=0.9
)

print(f"Grid size: {env.height}x{env.width}")
print(f"Number of states: {env.n_states}")
print(f"Number of actions: {env.n_actions}")
print(f"Goal state: {env.goal}")
print(f"Obstacles: {env.obstacles}")

## GridWorld Environment

We'll implement a classic GridWorld environment where:
- Agent moves in a 2D grid
- Goal state provides positive reward
- Obstacle states are blocked
- Each step has a small negative reward (to encourage efficiency)

## Markov Decision Process (MDP) Formulation

An MDP is defined by the tuple $(\mathcal{S}, \mathcal{A}, P, R, \gamma)$ where:

- $\mathcal{S}$: finite set of states
- $\mathcal{A}$: finite set of actions
- $P(s' | s, a)$: transition probability function
- $R(s, a, s')$: reward function
- $\gamma \in [0, 1)$: discount factor

The **state-value function** for policy $\pi$ is:

$$v_\pi(s) = \mathbb{E}_\pi[G_t | S_t = s] = \mathbb{E}_\pi\left[\sum_{k=0}^{\infty} \gamma^k R_{t+k+1} | S_t = s\right]$$

The **action-value function** is:

$$q_\pi(s, a) = \mathbb{E}_\pi[G_t | S_t = s, A_t = a]$$

The **Bellman equation** for $v_\pi$ is:

$$v_\pi(s) = \sum_a \pi(a|s) \sum_{s', r} p(s', r | s, a)[r + \gamma v_\pi(s')]$$